In [1]:
api_key = 'RGAPI-5f1da01a-8bc4-45fd-9eb1-c0f00e116249'

In [33]:
import pandas as pd
import requests
import cx_Oracle
from tqdm import tqdm

In [3]:
url_diamond_1 = 'https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/DIAMOND/I?page=1&api_key='+api_key

In [24]:
res_diamond_1 = requests.get(url_diamond_1).json()

In [5]:
len(res_diamond_1)

205

In [6]:
res_diamond_1[0].keys()

dict_keys(['leagueId', 'queueType', 'tier', 'rank', 'summonerId', 'summonerName', 'leaguePoints', 'wins', 'losses', 'veteran', 'inactive', 'freshBlood', 'hotStreak'])

In [7]:
summonerName_lst = list(map(lambda x: x['summonerName'], res_diamond_1))

In [8]:
summonerName_lst

['Sehee lu',
 '이블린 그 자체',
 '르 작',
 'Jasm1ne',
 '꿀태영',
 '냉동참치명치펀치',
 '싱거워',
 '갓병대 욱',
 '8분만 버티자',
 '김홍순3',
 '지겨워죽겠엉',
 '다시 태어난 애',
 '벡스 갈리오',
 '하이 눈 김주호',
 '겜포기안함',
 'SEK4',
 '괴롭히지마피즈',
 '신미야',
 '22 05 29bb',
 '블루와 옐로우',
 '아기도파',
 'Fatty',
 '씨맥의 동기부여',
 '나참챈싫다',
 '우리팀은 조평신',
 'Round One',
 '소영이는공주야',
 'mid revenant',
 '법무법인 김앤장',
 '구도비틀기',
 '새드보이제이비',
 '얼음 연꽃',
 'Pxxch',
 '겜에정떨어진사람',
 'SUPPOSSED',
 'Lovely',
 'SN Bin',
 '짜피나크면못막아',
 '지나가다멈춘부계',
 'zunyidiyi jug',
 '띠동갑남친',
 '오빠 이러지마',
 '서른웅',
 'Youtube Axiyo',
 '상 상',
 '돌망치 오우거',
 '쵸비루키페이커띵구쇼메이커비디디',
 '탑 라인전 최약체',
 '패배가무서운사람',
 '조까안가',
 '고 광',
 'lihai1',
 '빨간악보',
 'Esquif7',
 '마스터 우르곳',
 '트위치 카제',
 'moochin련',
 'Lv60 qiangshou',
 'Stay Sunday',
 '채팅차단하고홧팅',
 '유창하다',
 '미드는개백정',
 'Migung',
 '카운트다운4',
 '나는 너무 못한다',
 'itsuka Kotor1',
 '싸우지들마세여',
 '첼린저갈피지컬',
 '어둠잔fan',
 '카서스뽀삐쉔',
 'kiss and kill',
 '미라크리',
 '기백짱짱123',
 'skyplay',
 '어른도치씩씩',
 'llllIIllllllIIII',
 '난 바보양',
 '페이스북 정철환',
 'alalalalalala',
 'BVSide',
 '에 디',
 '아 뭔 미드근코여',
 '간단해요',

In [9]:
from my_utils import *

In [10]:
puuid = list(map(lambda x: get_puuid(x), summonerName_lst[:3]))

In [11]:
matchid = list(map(lambda x: get_matchid(x, 3), puuid))

In [12]:
matchs = []
for i in matchid:
    matchs.extend(get_matchs_timelines(i))

In [13]:
def get_lst(summonerName_lst):
    tmp = []
    for i in summonerName_lst[:3]:
        puuid = get_puuid(i)
        matchid = get_matchid(puuid, 3)
        matchs_timelines = get_matchs_timelines(matchid)
        tmp.extend(matchs_timelines)
    return tmp

In [27]:
tmp = get_lst(summonerName_lst)

In [29]:
df = pd.DataFrame(tmp, columns = ['gameid', 'matchs', 'timelines'])

In [30]:
df

,gameid,matchs,timelines
0,KR_5959757493,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
1,KR_5959723670,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
2,KR_5959674785,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
3,KR_5997104424,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
4,KR_5997030619,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
5,KR_5996994464,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
6,KR_6004039447,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
7,KR_6004006724,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
8,KR_6003971542,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."


In [34]:
def matchs_lst(df):
    lst = []
    for i in tqdm(range(len(df))):
        if df.iloc[i].matchs['info']['gameMode'] == 'CLASSIC':
            for j in range(10):
                try:
                    tmp = []
                    tmp.append(df.iloc[i].matchs['info']['gameId'])
                    tmp.append(df.iloc[i].matchs['info']['gameDuration'])
                    tmp.append(df.iloc[i].matchs['info']['gameVersion'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['participantId'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['teamId'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['championName'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['totalDamageDealt'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['totalDamageDealtToChampions'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['totalDamageTaken'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['teamPosition'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['summonerName'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['summonerLevel'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['kills'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['assists'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['deaths'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['summoner1Id'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['summoner2Id'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['item0'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['item1'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['item2'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['item3'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['item4'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['item5'])
                    tmp.append(df.iloc[i].matchs['info']['participants'][j]['item6'])
                    lst.append(tmp)
                except Exception as e:
                    print(e)
    return lst

In [36]:
tmp = matchs_lst(df)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 62.54it/s]


In [39]:
df = pd.DataFrame(tmp)
df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,5959757493,1689,12.11.445.6412,1,100,Graves,142549,6977,19602,TOP,...,6,12,4,6692,6676,1036,1036,2031,3047,3363
1,5959757493,1689,12.11.445.6412,2,100,Nidalee,140498,17258,29708,JUNGLE,...,7,11,4,3100,3108,0,3020,4636,1082,3364
2,5959757493,1689,12.11.445.6412,3,100,Yone,145738,17145,20452,MIDDLE,...,7,3,4,1054,3006,3033,6673,3133,1037,3364
3,5959757493,1689,12.11.445.6412,4,100,Seraphine,129066,11252,13552,BOTTOM,...,2,3,4,3040,0,6656,1011,3011,3158,3363
4,5959757493,1689,12.11.445.6412,5,100,Senna,37252,12510,13125,UTILITY,...,3,4,7,3864,6692,2055,3179,3086,3009,3364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,6003971542,1432,12.12.450.4196,6,200,Sejuani,94285,7902,15899,TOP,...,4,14,12,3070,1029,6662,3076,3047,1028,3364
76,6003971542,1432,12.12.450.4196,7,200,Graves,133526,11259,21130,JUNGLE,...,2,11,4,6630,2055,3133,1036,3111,3067,3364
77,6003971542,1432,12.12.450.4196,8,200,Cassiopeia,111655,19602,17087,MIDDLE,...,5,12,4,6656,2421,2055,3116,3040,1029,3364
78,6003971542,1432,12.12.450.4196,9,200,Ezreal,69853,8698,13713,BOTTOM,...,8,7,4,1055,3158,3078,3133,3070,1036,3363


In [14]:
division_lst = ['I', 'II', 'III', 'IV']

In [15]:
tier = 'BRONZE'

In [16]:
page = 1

In [17]:
def divi_lst(division_lst):
    lst = []
    for division in division_lst:
        url = 'https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/'+tier+'/'+division+'?page='+str(page)+'&api_key='+api_key
        res = requests.get(url).json()
        lst.extend(res)
    return lst

In [18]:
def get_rawData(tier):
    division_lst = ['I', 'II', 'III', 'IV']
    lst = []
    lst.extend(divi_lst(division_lst))
    summonerName_lst_tmp = []
    for i in range(len(lst)):
        summonerName_lst_tmp.append(lst[i]['summonerName'])
    data = []
    data.extend(get_lst(summonerName_lst_tmp))
    df = pd.DataFrame(data, columns = ['gameid', 'matchs', 'timelines'])
    return df

In [19]:
data_tmp = get_rawData(tier)

In [22]:
type(data_tmp)

pandas.core.frame.DataFrame

In [107]:
data_tmp.iloc[0].timelines['info']['frames'][0]['participantFrames']['1']['totalGold']

500

In [15]:
data_tmp.iloc[0].timelines['info'].keys()

dict_keys(['frameInterval', 'frames', 'gameId', 'participants'])

In [27]:
data_tmp.iloc[0].timelines['info']['participants'][0].keys()

dict_keys(['participantId', 'puuid'])

In [30]:
data_tmp.iloc[0].timelines['info']['frameInterval']

60000

In [191]:
data_tmp.iloc[0].timelines['info']['frames'][0]['participantFrames']['1']

{'championStats': {'abilityHaste': 0,
  'abilityPower': 0,
  'armor': 33,
  'armorPen': 0,
  'armorPenPercent': 0,
  'attackDamage': 25,
  'attackSpeed': 100,
  'bonusArmorPenPercent': 0,
  'bonusMagicPenPercent': 0,
  'ccReduction': 0,
  'cooldownReduction': 0,
  'health': 669,
  'healthMax': 669,
  'healthRegen': 0,
  'lifesteal': 0,
  'magicPen': 0,
  'magicPenPercent': 0,
  'magicResist': 32,
  'movementSpeed': 355,
  'omnivamp': 0,
  'physicalVamp': 0,
  'power': 251,
  'powerMax': 251,
  'powerRegen': 0,
  'spellVamp': 0},
 'currentGold': 500,
 'damageStats': {'magicDamageDone': 0,
  'magicDamageDoneToChampions': 0,
  'magicDamageTaken': 0,
  'physicalDamageDone': 0,
  'physicalDamageDoneToChampions': 0,
  'physicalDamageTaken': 0,
  'totalDamageDone': 0,
  'totalDamageDoneToChampions': 0,
  'totalDamageTaken': 0,
  'trueDamageDone': 0,
  'trueDamageDoneToChampions': 0,
  'trueDamageTaken': 0},
 'goldPerSecond': 0,
 'jungleMinionsKilled': 0,
 'level': 1,
 'minionsKilled': 0,
 'pa

In [207]:
data_tmp.iloc[7].timelines['info']['frames'][0]['participantFrames']['10']['totalGold']

500

In [165]:
len(data_tmp.iloc[0].timelines['info']['frames'])

20

In [164]:
len(data_tmp.iloc[0].timelines['info']['frames'][19]['participantFrames'])

10

In [148]:
len(data_tmp.iloc[0].timelines['info']['frames'][1]['events'])

38

In [151]:
data_tmp.iloc[0].timelines['info']['frames'][1]['events'][37]

{'levelUpType': 'NORMAL',
 'participantId': 1,
 'skillSlot': 1,
 'timestamp': 52313,
 'type': 'SKILL_LEVEL_UP'}

In [20]:
data_tmp.iloc[0].matchs['info'].keys()

dict_keys(['gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'participants', 'platformId', 'queueId', 'teams', 'tournamentCode'])

In [21]:
data_tmp.iloc[0].matchs['info']['gameMode']

'CLASSIC'

In [181]:
len(data_tmp.iloc[0].matchs['info']['participants'])

10

In [202]:
data_tmp.iloc[0].matchs['info']['participants'][1]['item6'] #장신구

3340

In [18]:
data_tmp.iloc[0].matchs['info']['participants'][0].keys()

dict_keys(['assists', 'baronKills', 'bountyLevel', 'challenges', 'champExperience', 'champLevel', 'championId', 'championName', 'championTransform', 'consumablesPurchased', 'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets', 'damageSelfMitigated', 'deaths', 'detectorWardsPlaced', 'doubleKills', 'dragonKills', 'eligibleForProgression', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist', 'firstTowerKill', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'goldEarned', 'goldSpent', 'individualPosition', 'inhibitorKills', 'inhibitorTakedowns', 'inhibitorsLost', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'itemsPurchased', 'killingSprees', 'kills', 'lane', 'largestCriticalStrike', 'largestKillingSpree', 'largestMultiKill', 'longestTimeSpentLiving', 'magicDamageDealt', 'magicDamageDealtToChampions', 'magicDamageTaken', 'neutralMinionsKilled', 'nexusKills', 'nexusLost', 'nexusTakedowns', 'objectivesStolen', 'objectivesStolenAssists', 'parti

In [37]:
data_tmp.iloc[0].matchs['info']['participants'][0]['summoner2Id']

4

In [38]:
data_tmp.iloc[0].matchs['info']['participants'][0]['itemsPurchased']

2

In [36]:
from tqdm import tqdm

In [119]:
def matchs_lst(df):
    lst = []
    for i in tqdm(range(len(df))):
        for j in range(len(df.iloc[i].matchs['info']['participants'])):
            tmp = []
            tmp.append(df.iloc[i].matchs['info']['gameId'])
            tmp.append(df.iloc[i].matchs['info']['gameDuration'])
            tmp.append(df.iloc[i].matchs['info']['gameVersion'])
            tmp.append(df.iloc[i].matchs['info']['participants'][j]['participantId'])
            tmp.append(df.iloc[i].matchs['info']['participants'][j]['teamId'])
            tmp.append(df.iloc[i].matchs['info']['participants'][j]['championName'])
            tmp.append(df.iloc[i].matchs['info']['participants'][j]['totalDamageDealt'])
            tmp.append(df.iloc[i].matchs['info']['participants'][j]['totalDamageDealtToChampions'])
            tmp.append(df.iloc[i].matchs['info']['participants'][j]['totalDamageTaken'])
            tmp.append(df.iloc[i].matchs['info']['participants'][j]['teamPosition'])
            tmp.append(df.iloc[i].matchs['info']['participants'][j]['summonerName'])
            tmp.append(df.iloc[i].matchs['info']['participants'][j]['summonerLevel'])
            lst.append(tmp)
    return lst

In [120]:
matchs = matchs_lst(data_tmp)

100%|███████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 138.55it/s]


In [121]:
def timelines_lst(df):
    lst = []
    for i in tqdm(range(len(df))):
        for j in range(1, len(df.iloc[0].timelines['info']['frames'][0]['participantFrames'])+1):
            tmp = []
            tmp.append(df.iloc[i].timelines['info']['gameId'])
            tmp.append(df.iloc[i].timelines['info']['participants'][j-1]['participantId'])
            tmp.append(df.iloc[i].timelines['info']['frames'][5]['participantFrames'][str(j)]['totalGold'])
            tmp.append(df.iloc[i].timelines['info']['frames'][6]['participantFrames'][str(j)]['totalGold'])
            tmp.append(df.iloc[i].timelines['info']['frames'][7]['participantFrames'][str(j)]['totalGold'])
            lst.append(tmp)
    return lst

In [122]:
timelines = timelines_lst(data_tmp)

100%|███████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 257.30it/s]


In [123]:
df_matchs = pd.DataFrame(matchs, columns = ['gameId', 'gameDuration', 'gameVersion', 'participantId', 'teamId', 
                                            'championName', 'totalDamageDealt', 'totalDamageDealtToChampions', 
                                            'totalDamageTaken', 'teamPosition', 'summonerName', 'summonerLevel'])
df_timelines = pd.DataFrame(timelines, columns = ['gameId', 'participantId', 'g_5', 'g_6', 'g_7'])

In [124]:
merge_df = pd.merge(df_matchs, df_timelines)

In [126]:
db_open()

db_open!


In [134]:
query = 'drop table lol_stats'

In [136]:
query = """
create table lol_stats(gameId varchar(20), gameDuration varchar(20), gameVersion varchar(20), participantId varchar(20), teamId varchar(20), championName varchar(20), totalDamageDealt number(20), totalDamageDealtToChampions number(20), totalDamageTaken number(20),
teamPosition varchar(30), summonerName varchar(30), summonerLevel varchar(20), g_5 number(20), g_6 number(20), g_7 number(20), constraint match_timeline_pk primary key (gameId, participantId))
"""

In [137]:
sql_execute(query)

'success'

In [140]:
def insert(x):
    query = (f"insert into lol_stats(gameId, gameDuration, gameVersion, participantId, teamId, totalDamageDealt, totalDamageTaken, teamPosition, summonerName, summonerLevel, g_5, g_6, g_7)"
            f"values(\'{x.gameId}\', \'{x.gameDuration}\', \'{x.gameVersion}\', \'{x.participantId}\', \'{x.teamId}\', \'{x.championName}\', {x.totalDamageDealt}, {x.totalDamageDealtToChampions}, {x.totalDamageTaken},"
            f"\'{x.teamPosition}\', \'{x.summonerName}\', \'{x.summonerLevel}\', {x.g_5}, {x.g_6}, {x.g_7})")
    sql_execute(query)
    return

In [141]:
tqdm.pandas()
merge_df.progress_apply(lambda x: insert(x), axis = 1)

100%|█████████████████████████████████████████████████████████████████████████████████| 90/90 [00:00<00:00, 331.65it/s]


0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [142]:
query = 'select * from lol_stats'

In [143]:
sql_execute(query)

C:\Users\user\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,GAMEID,GAMEDURATION,GAMEVERSION,PARTICIPANTID,TEAMID,TOTALDAMAGEDEALT,TOTALDAMAGETAKEN,TEAMPOSITION,SUMMONERNAME,SUMMONERLEVEL,G_5,G_6,G_7
0,5938416886,1934,12.10.442.9993,1,100,107997,40925,JUNGLE,snowman0222,151,1808,2062,2253
1,5938416886,1934,12.10.442.9993,2,100,121864,33370,TOP,알파쿤,64,1817,2031,2188
2,5938416886,1934,12.10.442.9993,3,100,91953,41150,BOTTOM,탑애메,82,1126,1653,1864
3,5938416886,1934,12.10.442.9993,4,100,7896,4920,UTILITY,강형욱네보신탕집,55,955,1339,1473
4,5938416886,1934,12.10.442.9993,5,100,163783,18715,MIDDLE,쉐도우베라럭스,183,1529,1746,2275
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,5992818535,1779,12.12.450.4196,6,200,220240,29660,JUNGLE,vaemqrlcxvz,123,1485,1792,2085
86,5992818535,1779,12.12.450.4196,7,200,127907,19070,MIDDLE,떡볶이개꿀맛,154,1729,1988,2740
87,5992818535,1779,12.12.450.4196,8,200,151034,28468,TOP,꽃미남 우르곳,34,1403,2010,2132
88,5992818535,1779,12.12.450.4196,9,200,65104,12767,BOTTOM,준비취소,143,1487,1924,2343


In [144]:
db_close()

db_close!
